In [3]:
import numpy as np
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from keras.models import Model

# Define Conv2d_BN function
def Conv2d_BN(x, filters, kernel_size, strides=(1, 1), padding='same'):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# Define Residual Block function
def Residual_Block(x, nb_filter, kernel_size=(3, 3), strides=(1, 1), with_conv_shortcut=False):
    shortcut = x

    x = Conv2d_BN(x, nb_filter, kernel_size, strides=strides)
    x = Conv2d_BN(x, nb_filter, kernel_size)

    if with_conv_shortcut:
        shortcut = Conv2d_BN(shortcut, nb_filter, (1, 1), strides=strides, padding='valid')

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Define the ResNet34 model
def ResNet34(width, height, depth, classes):
    Img = Input(shape=(width, height, depth))

    x = Conv2d_BN(Img, 64, (7, 7), strides=(2, 2), padding='same')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Residual blocks
    x = Residual_Block(x, nb_filter=64, kernel_size=(3, 3))
    x = Residual_Block(x, nb_filter=64, kernel_size=(3, 3))
    x = Residual_Block(x, nb_filter=64, kernel_size=(3, 3))

    x = Residual_Block(x, nb_filter=128, kernel_size=(3, 3), strides=(2, 2), with_conv_shortcut=True)
    for _ in range(3):
        x = Residual_Block(x, nb_filter=128, kernel_size=(3, 3))

    x = Residual_Block(x, nb_filter=256, kernel_size=(3, 3), strides=(2, 2), with_conv_shortcut=True)
    for _ in range(5):
        x = Residual_Block(x, nb_filter=256, kernel_size=(3, 3))

    x = Residual_Block(x, nb_filter=512, kernel_size=(3, 3), strides=(2, 2), with_conv_shortcut=True)
    for _ in range(2):
        x = Residual_Block(x, nb_filter=512, kernel_size=(3, 3))

    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=Img, outputs=x)
    return model

# Example usage
if __name__ == "__main__":
    # Create the model
    model = ResNet34(224, 224, 3, 10)  # Example: 224x224 RGB images, 10 classes

    # Print the model summary
    model.summary()

    # Generate dummy input data (e.g., a batch of 8 images)
    input_data = np.random.rand(8, 224, 224, 3)  # Batch of 8 images

    # Get predictions
    predictions = model.predict(input_data)
    print("Output shape:", predictions.shape)  # Should be (8, 10)
    print("Predictions:", predictions)  # Softmax probabilities for each class


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_36 (Conv2D)        │ (None, 112, 112, 64)   │          9,472 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_36    │ (None, 112, 112, 64)   │            256 │ conv2d_36[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 112, 112, 64)   │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 56, 56, 64)     │              0 │ activation[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_37 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_37    │ (None, 56, 56, 64)     │            256 │ conv2d_37[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 56, 56, 64)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_38 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_38    │ (None, 56, 56, 64)     │            256 │ conv2d_38[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 56, 56, 64)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_16 (Add)              │ (None, 56, 56, 64)     │              0 │ activation_2[0][0],    │
│                           │                        │                │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 56, 56, 64)     │              0 │ add_16[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_39 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_39    │ (None, 56, 56, 64)     │            256 │ conv2d_39[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activation) │ (None, 56, 56, 64)     │              0 │ batch_normalization_3… │
├──────────────────────

 Total params: 21,315,338 (81.31 MB)

 Trainable params: 21,298,314 (81.25 MB)

 Non-trainable params: 17,024 (66.50 KB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Output shape: (8, 10)
Predictions: [[3.91867980e-02 1.27756055e-02 1.55191636e-02 1.40897534e-03
  9.06945705e-01 1.68710935e-03 8.35037883e-03 6.95330603e-03
  3.05599883e-07 7.17262225e-03]
 [3.83987837e-02 1.28952172e-02 1.54464971e-02 1.39228895e-03
  9.07654464e-01 1.68075413e-03 8.40802398e-03 6.99361833e-03
  3.08227442e-07 7.13012693e-03]
 [3.86447832e-02 1.27257956e-02 1.51768597e-02 1.38977938e-03
  9.07609642e-01 1.67993712e-03 8.52680486e-03 6.96391007e-03
  3.13965813e-07 7.28215883e-03]
 [3.88203971e-02 1.28985560e-02 1.55850919e-02 1.42630551e-03
  9.06807005e-01 1.69465865e-03 8.34576320e-03 7.17327278e-03
  3.09701136e-07 7.24868057e-03]
 [3.89195532e-02 1.27211427e-02 1.54822282e-02 1.41733629e-03
  9.07259226e-01 1.65780063e-03 8.44077673e-03 6.98023057e-03
  3.07234700e-07 7.12139672e-03]
 [3.87595519e-02 1.28081795e-02 1.56790130e-02 1.40183535e-03
  9.07022476e-01 1.66130520e-03 8.52001272e-03 7.03168940e-03
  2.98736467e-07 7.1